# Prov-GigaPath inference demo

The Prov-GigaPath models can be accessed from [HuggingFace Hub](https://huggingface.co/prov-gigapath/prov-gigapath).

You need to agree to the terms to access the models. Once you have the necessary access, set your HuggingFace read-only token as an environment variable:

In [1]:
import os
os.environ["HF_TOKEN"] = "<HuggingFace Token>"

## Prepare image tiles

To facilitate the demo, you can access and download preprocessed tiles from this link: https://hanoverprod.z21.web.core.windows.net/gigapath/PANDA_sample_tiles.zip

The zip file includes a sample slide we processed from the PANDA dataset.

In [2]:
slide_dir = "data/PANDA_sample_tiles/054b6888604d963455bfff551518ece5/"
image_paths = [os.path.join(slide_dir, img) for img in os.listdir(slide_dir) if img.endswith('.png')]

print(f"Found {len(image_paths)} image tiles")

Found 810 image tiles


## Load the tile and slide encoder model separately

In [ ]:

import timm
import gigapath.slide_encoder as slide_encoder


# load the tile encoder
tile_encoder = timm.create_model("hf_hub:prov-gigapath/prov-gigapath", pretrained=True)
# load the slide encoder
slide_encoder_model = slide_encoder.create_model("hf_hub:prov-gigapath/prov-gigapath", "gigapath_slide_enc12l768d", 1536)

## Load both the tile and slide encoder model at once

In [3]:
from gigapath.pipeline import load_tile_slide_encoder
# load the tile and slide encoder models
# Note: Older versions of timm have compatibility issues. 
# Please ensure that you use a newer version by running the following command: pip install timm>=1.0.3.
tile_encoder, slide_encoder_model = load_tile_slide_encoder()

/home/naotous/miniconda3/envs/run_gigapath/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tile encoder param # 1134953984
dilated_ratio:  [1, 2, 4, 8, 16]
segment_length:  [1024, 5792, 32768, 185363, 1048576]
Number of trainable LongNet parameters:  85148160
Global Pooling: False


slide_encoder.pth: 100%|██████████| 345M/345M [00:00<00:00, 366MB/s] 


 Successfully Loaded Pretrained GigaPath model from hf_hub:prov-gigapath/prov-gigapath 
Slide encoder param # 86330880


## Run inferrence with tile encoder

In [4]:
from gigapath.pipeline import run_inference_with_tile_encoder
# run inference with the tile encoder
tile_encoder_outputs = run_inference_with_tile_encoder(image_paths, tile_encoder)

for k in tile_encoder_outputs.keys():
    print(f"tile_encoder_outputs[{k}].shape: {tile_encoder_outputs[k].shape}")

Running inference with tile encoder: 100%|██████████| 7/7 [00:07<00:00,  1.06s/it]

tile_encoder_outputs[tile_embeds].shape: torch.Size([810, 1536])
tile_encoder_outputs[coords].shape: torch.Size([810, 2])


## Run inference with the slide encoder

In [5]:
from gigapath.pipeline import run_inference_with_slide_encoder
# run inference with the slide encoder
slide_embeds = run_inference_with_slide_encoder(slide_encoder_model=slide_encoder_model, **tile_encoder_outputs)
print(slide_embeds.keys())

dict_keys(['layer_0_embed', 'layer_1_embed', 'layer_2_embed', 'layer_3_embed', 'layer_4_embed', 'layer_5_embed', 'layer_6_embed', 'layer_7_embed', 'layer_8_embed', 'layer_9_embed', 'layer_10_embed', 'layer_11_embed', 'layer_12_embed', 'last_layer_embed'])


## Additional notes

In this demo, we provided a walkthrough of running the pretrained Prov-GigaPath model. For instructions on fine-tuning the model, please refer to the following link: [Fine-Tuning Instructions](https://github.com/prov-gigapath/prov-gigapath?tab=readme-ov-file#fine-tuning)